In [ ]:
# this cell mount google drive - REQUEST PERMISSION!
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import numpy as np
import pandas as pd
import pathlib
import os
import cv2
from pathlib import Path
from google.colab.patches import cv2_imshow
import json
import matplotlib.pyplot as plt
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.layers import RandomRotation, RandomFlip, RandomContrast, Resizing
from tensorflow.image import flip_left_right, adjust_contrast, random_brightness, adjust_hue, adjust_saturation, random_crop
from tensorflow.keras.preprocessing.image import random_shift, random_zoom

In [ ]:
df = pd.read_csv("/content/gdrive/MyDrive/df_total_downloaded.csv" , converters={'video_id': str})

In [ ]:
def get_ids(gloss):
  return list(df.video_id[df.gloss == gloss])

In [ ]:
glist_top50 = list(df.gloss.value_counts()[:50].index)

In [ ]:
# Augmentation funtions

# adjust contrast
con_img = adjust_contrast(img, 2).numpy()

# flip image
rev_img = flip_left_right(img).numpy()

# adjust brightness
bright_img = random_brightness(img, 0.8).numpy()

# zoom-in image
crop_img = random_zoom(img, (0.85, 0.85), row_axis=1, col_axis=0, channel_axis=2)

# adjust saturation
satur_img = adjust_saturation(img, 1.5).numpy()

#  adjust hue
hue_img = adjust_hue(img, 0.05).numpy()

# shift horizontally
shift_img = random_shift(img, 0.3, 0, row_axis=1, col_axis=0, channel_axis=2)

# rotate image
rot_img = RandomRotation(factor=0.1, fill_mode='constant')(img).numpy().astype("uint8")

In [ ]:
def list_files_in_dir(path):

  # create list with all files in directory

  return [f for f in os.listdir(path) if 
               os.path.isfile(os.path.join(path, f))]

In [ ]:
def augment_images(directories_paths):
  """
    creates augmentations for all images in specified dirs.
  
    Parameters:
    directories_paths (lsit): paths to the dirs. with images to augment
    """
  # iterate over all directories given
  for dir in directories_paths:

    # iterate over all images in dir
    for img_path in list_files_in_dir(dir):

      # load image
      img = cv2.imread(dir + '/' + img_path)

      # augment
      con_img = adjust_contrast(img, 2).numpy()
      rev_img = flip_left_right(img).numpy()
      bright_img = random_brightness(img, 0.8).numpy()
      crop_img = random_zoom(img, (0.85, 0.85), row_axis=1, col_axis=0, channel_axis=2)
      satur_img = adjust_saturation(img, 1.5).numpy()
      hue_img = adjust_hue(img, 0.05).numpy()
      shift_img = random_shift(img, 0.3, 0, row_axis=1, col_axis=0, channel_axis=2)
      rot_img = RandomRotation(factor=0.1, fill_mode='constant')(img).numpy().astype("uint8")

      augment_list = [rot_img, shift_img, rev_img, crop_img, satur_img, hue_img, bright_img, con_img]
      augment_names = ['rotated', 'shifted', 'flipped', 'zoom-in', 'saturated', 'hue', 'brightness', 'contrast']
      
      # write augments to directory
      for i, image in enumerate(list(zip(augment_list, augment_names))):
        path = dir + '/' + img_path.split('.')[0] + f'_{image[1]}' + '.jpeg'
        print(path)
        cv2.imwrite(path, image[0])


In [ ]:
def create_paths_to_augment(base_path):
  
  # creates a list of all directories to be augmented in base dir.

  subset_dirs = os.listdir(base_path)
  return_paths = []
  # for dir in subset_dirs:
  for dir in ['train']:
    classes = os.listdir(base_path + '/' + dir)
    for c in classes:
      return_paths.append(base_path + '/' + dir + '/' + c)

  return return_paths

In [ ]:
dirs_to_augment = create_paths_to_augment('/content/gdrive/MyDrive/dataset/top50_full')

In [ ]:
augment_images(dirs_to_augment)